In [1]:
from project1_model import ResNet, BasicBlock 
import numpy as np, torch, torchvision 

In [2]:
trainingdata = torchvision.datasets.CIFAR10('./CIFAR10/', 
                                                 train=True, 
                                                 download=True, 
                                                 transform=torchvision.transforms.ToTensor()) 

testdata = torchvision.datasets.CIFAR10('./CIFAR10/', 
                                             train=False, 
                                             download=True, 
                                             transform=torchvision.transforms.ToTensor()) 


0.0%

1.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

7.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

12.4%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

18.5%IOPub m

Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10/
Files already downloaded and verified


In [3]:
print(len(trainingdata))
print(len(testdata))

trainDataLoader = torch.utils.data.DataLoader(trainingdata, 
                                              batch_size=64, 
                                              shuffle=True)

testDataLoader = torch.utils.data.DataLoader(testdata, 
                                             batch_size=64, 
                                             shuffle=False) 


50000
10000


In [4]:
images, labels = iter(trainDataLoader).next() 

In [5]:
net = ResNet(BasicBlock, [2, 2, 2, 2]) 
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [ ]:
train_loss_history = []
test_loss_history = []

for epoch in range(2):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(trainDataLoader):
        images, labels = data
        images = images 
        labels = labels 
        optimizer.zero_grad()
        predicted_output = net(images)
        fit = Loss(predicted_output,labels)
        fit.backward()
        optimizer.step()
        train_loss += fit.item()
    for i, data in enumerate(testDataLoader):
        with torch.no_grad():
            images, labels = data
            images = images 
            labels = labels 
            predicted_output = net(images)
            fit = Loss(predicted_output,labels)
            test_loss += fit.item()
    train_loss = train_loss/len(trainDataLoader)
    test_loss = test_loss/len(testDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s, Train loss %s, Test loss %s'%(epoch, train_loss, test_loss))